In [1]:
from cumulant import csolve

In [19]:
!pip install qutip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 5.2 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for qutip: filename=qutip-4.7.5-cp312-cp312-linux_x86_64.whl size=6520619 sha256=6dcdb49d72127c80c1f17b2701737b8245a7235733c604c5034e85983a846c16
  Stored in directory: /home/mcditoos/.cache/pip/wheels/8c/5f/9e/f9ccbe178ac2e6b68831ee4df56149c4b6e1c6b917cee5578e
Successfully built qutip


In [3]:
import numpy as np

In [5]:
w0 = 1
Deltaw = 1e-2*w0
alpha = 0.000159155*w0
gamma = 5*w0
T = 1*w0
tf = 40000
Hsys = np.array([[w0+(Deltaw/2), 0, 0], [0, w0-(Deltaw/2), 0], [0, 0, 0]])
Q = np.array([[0,0,1],[0,0,1],[1,1,0]])

In [6]:
cc = csolve(np.array([[1,0],[0,-1]]),np.linspace(0,tf,5000) , 1e-6, alpha, gamma, T, np.array([[0,1],[1,0]]), "ohmic")

In [8]:
rho0=np.eye(3)/3

In [11]:
sz=np.array([[1,0],[0,-1]])

In [13]:
evals, all_state = np.linalg.eig(sz)

In [18]:
all_state

array([[1., 0.],
       [0., 1.]])

In [15]:
evals

array([ 1., -1.])

In [9]:
eldict=cc.generator()

AttributeError: 'numpy.ndarray' object has no attribute 'eigenstates'

In [11]:
from qutip import spre,spost

In [12]:
sz=sigmaz().full()
right=np.kron(sz,np.eye(2))

In [13]:
sx=sigmax().full()

In [14]:
sigmaz()*sigmax()

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = False
Qobj data =
[[ 0.  1.]
 [-1.  0.]]

In [15]:
spre(sigmaz())(sigmax())

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = False
Qobj data =
[[ 0.  1.]
 [-1.  0.]]

In [16]:
(right@sx.reshape(4)).reshape((2,2))

array([[ 0.+0.j,  1.+0.j],
       [-1.+0.j,  0.+0.j]])

In [17]:
Qobj(right)

Quantum object: dims = [[4], [4]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  0.  0. -1.]]

In [18]:

def spre_np(op):
    right = np.kron(op,np.eye(op.shape[0]))
    return right#lambda x:(right@x.reshape(op.shape[0]**2)).reshape(op.shape[0],op.shape[0])
def spost_np(op):
    left=np.kron(np.eye(op.shape[0]),op.T)
    return left#lambda x: (left@x.reshape(op.shape[0]**2)).reshape(op.shape[0],op.shape[0])

In [144]:
class spre_np:
    def __init__(self, op):
        self.right = np.kron(op,np.eye(op.shape[0]))
        self.dim=op.shape[0]
        self.func=lambda x:(self.right@x.reshape(op.shape[0]**2)).reshape(op.shape[0],op.shape[0])
    def __call__(self, *args, **kwargs):
        return self.func(*args,**kwargs)
    def __add__(self, other):
        self.right+=other.right
        self.func=lambda x:(self.right@x.reshape(self.dim**2)).reshape(self.dim,self.dim)
        return self

    def __sub__(self, other):
        self.right -= other.right
        self.func = lambda x: (
            self.right@x.reshape(self.dim**2)).reshape(self.dim, self.dim)
        return self

    def __mul__(self, other):
        if type(other) in (int, float, complex):
            self.right *= other
            self.func = lambda x: (
                self.right@x.reshape(self.dim**2)).reshape(self.dim, self.dim)
            return self
        self.right = self.right @ other.right
        self.func = lambda x: (
            self.right@x.reshape(self.dim**2)).reshape(self.dim, self.dim)
        return self
    
class spost_np:
    def __init__(self, op):
        self.right = np.kron(np.eye(op.shape[0]), op.T)
        self.dim=op.shape[0]
        self.func = lambda x: (
            self.right@x.reshape(op.shape[0]**2)).reshape(op.shape[0], op.shape[0])

    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs)

    def __add__(self, other):
        self.right += other.right
        self.func = lambda x: (
        self.right@x.reshape(self.dim**2)).reshape(self.dim, self.dim)
        return self
    def __sub__(self, other):
        self.right -= other.right
        self.func = lambda x: (
        self.right@x.reshape(self.dim**2)).reshape(self.dim, self.dim)
        return self
    def __mul__(self, other):
        if type(other) in (int,float,complex):
            self.right*=other
            self.func = lambda x: (
            self.right@x.reshape(self.dim**2)).reshape(self.dim, self.dim)
            return self
        self.right =self.right@ other.right
        self.func = lambda x: (
        self.right@x.reshape(self.dim**2)).reshape(self.dim, self.dim)
        return self
        

In [142]:
type(sx)

numpy.ndarray

In [99]:
i=[-2,2]

In [100]:
expr=spre(eldict[i[1]]) * spost(eldict[i[0]].dag()) 

In [101]:
expr(sigmay())

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = False
Qobj data =
[[0.+0.j 0.+0.j]
 [0.-1.j 0.+0.j]]

In [102]:
spre_np(sx)

In [103]:
expr = spre_np(eldict[i[1]].full()) * spost_np(eldict[i[0]].dag().full())

In [104]:
expr(sigmay().full())

array([[0.+0.j, 0.+0.j],
       [0.-1.j, 0.+0.j]])

In [118]:

ss= spre(eldict[i[0]].dag() * eldict[i[1]]) + spost(eldict[i[0]].dag() * eldict[i[1]])

In [132]:
ss_np = spre_np(eldict[i[0]].dag().full() @ eldict[i[1]].full()) + spost_np(eldict[i[0]].dag().full() @ eldict[i[1]].full())
ss_np

In [134]:
ss_np(sz)

array([[0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j]])

In [135]:
ss(sigmaz())

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0. 0.]
 [0. 0.]]

In [ ]:
(spre(eldict[i[1]]) * spost(eldict[i[0]].dag()) -
 (0.5 *
  (spre(eldict[i[0]].dag() * eldict[i[1]]) + spost(
      eldict[i[0]].dag() * eldict[i[1]]))))